In [13]:
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
import keras
import tensorflow as tf
from keras import backend as K
warnings.filterwarnings('ignore')
import pandas as pd
from pylab import *

# 1. Training

In [7]:
# load data
trainset = pd.read_pickle('../data/pickle/dataset_LAYERS_SUMCLUSTERS_train.pkl')
x_train,y_train = [],[]
for i in range(len(trainset)):
    temp = trainset.loc[i].feature
    temp = np.array(temp).reshape(-1)
    x_train.append(temp)
    
    l = trainset.loc[i].label
    temp = np.zeros(4)
    temp[l] = 1
    y_train.append(temp)
    
x_train = np.array(x_train)[:,None,:]
y_train = np.array(y_train)

# load model
from keras.layers import Dense,Conv1D,Flatten,Input
from keras import Model
def shower_classification_model():
    input_img = Input(shape=(1, 52), name='input')
    conv = Conv1D(4, 5, activation='relu', padding='same', data_format='channels_first', name='conv1')(input_img)
    conv = Conv1D(4, 4, activation='relu', padding='same', data_format='channels_first', name='conv2')(conv)
    conv = Conv1D(4, 4, activation='relu', padding='same', data_format='channels_first', name='conv3')(conv)
    flat = Flatten()(conv)
    dense = Dense(16, activation='relu', input_dim=80, name='dense1')(flat)
    pred = Dense(4, activation='softmax', name='output')(dense)
    model = Model(inputs=input_img, outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = shower_classification_model()

In [8]:
# train
model.fit(x_train, y_train, epochs=10, batch_size=100,validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 27000 samples, validate on 3000 samples
Epoch 1/10
27000/27000 [==============================] - 2s 68us/step - loss: 0.5836 - acc: 0.7244 - val_loss: 0.4365 - val_acc: 0.7493
Epoch 2/10
27000/27000 [==============================] - 1s 47us/step - loss: 0.4172 - acc: 0.7648 - val_loss: 0.4215 - val_acc: 0.7767
Epoch 3/10
27000/27000 [==============================] - 1s 49us/step - loss: 0.4076 - acc: 0.7711 - val_loss: 0.4110 - val_acc: 0.7820
Epoch 4/10
27000/27000 [==============================] - 1s 45us/step - loss: 0.3998 - acc: 0.7750 - val_loss: 0.4068 - val_acc: 0.7813
Epoch 5/10
27000/27000 [==============================] - 1s 44us/step - loss: 0.3961 - acc: 0.7760 - val_loss: 0.4057 - val_acc: 0.7830
Epoch 6/10
27000/27000 [==============================] - 1s 46us/step - loss: 0.3936 - acc: 0.7771 - val_loss: 0.4050 - val_acc: 0.7780
Epoch 7/10
27000/27000 [==============================] - 1s 48us/step - loss: 0.

In [14]:
# save model
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph
    
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

tf.train.write_graph(frozen_graph, "../models/", "CNN.pb", as_text=False)
tf.train.write_graph(frozen_graph, "../models/", "CNN.pbtxt", as_text=True)

INFO:tensorflow:Froze 60 variables.
INFO:tensorflow:Converted 60 variables to const ops.


'../models/CNN.pbtxt'